In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pickle
import pandas as pd
import numpy as np
from copy import copy
import tensorflow as tf
from sklearn.model_selection import train_test_split

import pymedtermino
import seaborn as sns

from metric_learn import NCA
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsClassifier

from tpot import TPOTClassifier

In [3]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer

import sys
sys.path.append(f'../../')

from src.data.sentence_vectorizer import SentenceVectorizer

In [4]:
cadec_train = pd.read_csv('../../data/interim/cadec/train.csv')
cadec_test = pd.read_csv('../../data/interim/cadec/test.csv')
cadec_train.shape, cadec_test.shape

((3520, 8), (881, 8))

In [5]:
sentvec = SentenceVectorizer()
train, test = sentvec.vectorize_sent_tfidf(cadec_train, cadec_test, feat_col='term', 
                                           text_columns = ['term', 'text', 'SNMS']) #'sent', 'text', 'STR', 'SNMS'

In [6]:
train

,term,start,end,sent,text,code,STR,SNMS,term_vec
0,Sight loss,173,183,Have a 5 month supply but getting off quick!.,"slurring in speech, lower back pain, bloated, ...",10047571,Visual impairment,"['Visual impairment', 'Visual impairment', 'Vi...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,gas pains in abdomen,47,54,It lowered my LDL (not enough) but also lowere...,Terrific cramps in legs and awful gas pains in...,10016767,"Flatulence, eructation, and gas pain","['Flatulence, eructation and gas pain', '[D]Fl...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8079093530333..."
2,knees hurt,692,696,Doctor is PISSED-----I'M THRILLED !!!.,Over the course of the past year I've experien...,10023477,Knee pain,"['Knee pain', 'Knee pain', 'Knee pain', 'Knee ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,EXTREME nausea,497,511,vomitting - I have to fight to not throw up I ...,vomitting - I have to fight to not throw up I ...,10028813,Nausea,"['Nausea', 'Nausea', 'Nausea', 'Nausea', 'Naus...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,frequently unable to think clearly,110,144,"So far my blood tests have been normal, but no...","After three months on Lipitor, I started to fe...",10012805,Difficulty thinking,[],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...,...,...,...,...,...,...
3515,joint stiffness,21,36,I didn't realize this was the cause until I re...,"Headaches,body aches,joint stiffness,bloating,...",10023230,Joint stiffness,"['Joint stiffness', 'Joint stiffness NOS', 'Jo...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3516,gas,1244,1247,He took Tums to alliviate stomach upsets and a...,I used to take Provachol (sp?)but I found it c...,10016766,Flatulence,"['Flatulence', 'Flatulence', 'Flatulence', 'Fl...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3517,stomach pain,424,436,Now I have to see my doctor again to see if I ...,"For the first 8 days of ever taking it, the on...",10042076,Stomach ache,"['Stomach ache', 'Stomach ache', 'Belly ache',...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3518,Severe elbow pain,31,35,"Severe elbow, arm and shoulder pain.","Severe elbow, arm and shoulder pain.<SENT>It's...",10033371,Pain,"['Pain', 'Pain', 'Pain', 'Pain', 'Pain', 'Pain...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [7]:
X_train, y_train = train['term_vec'], train['code']
X_test, y_test = test['term_vec'], test['code']

X_train = pd.DataFrame([pd.Series(x) for x in X_train])
X_test = pd.DataFrame([pd.Series(x) for x in X_test])

In [8]:
labels = pd.concat([y_train, y_test])
labels.shape

(4401,)

In [9]:
meddra_labels = {v:k for k, v in enumerate(labels.unique())}
y_train = y_train.apply(lambda x: int(meddra_labels[x]))
y_test = y_test.apply(lambda x: int(meddra_labels[x]))

In [10]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((3520, 400), (3520,), (881, 400), (881,))

In [113]:
generations = 2
population_size = 50
max_eval_time_mins = 2
n_jobs = 10
max_iter = 10

tpot = TPOTClassifier(generations=generations, 
                      population_size=population_size,
                      verbosity=2, 
                      random_state=42, 
                      max_eval_time_mins=max_eval_time_mins, 
                      n_jobs=n_jobs)

tpot.fit(X_train, y_train)
score = tpot.score(X_test, y_test)
logging.debug(f'PURE MEDDRA: {score}')

Optimization Progress:   0%|          | 0/150 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.4315340909090909

Generation 2 - Current best internal CV score: 0.4326704545454545

Best pipeline: LinearSVC(input_matrix, C=5.0, dual=False, loss=squared_hinge, penalty=l2, tol=1e-05)


NameError: name 'logging' is not defined

In [114]:
score

0.4245175936435868

In [115]:
1+1

2

SVC(gamma='auto')

In [14]:
clf.score(X, y)

1.0